In [33]:
import random
from copy import deepcopy

In [34]:
STOCK_SIZE = 100

In [35]:
def read_instance(path):
    with open(path, 'r') as f:
        orders = [int(x) for x in f.readline().split()]
        return orders

In [36]:
# 1 * w[0] + 2 * w[1] + .... + M * w[M-1]

def calc_solution_value(orders, stocks): 

    leftovers = deepcopy(stocks)
    
    for order in orders:
        for j, leftover_size in enumerate(leftovers):
            if order <= leftover_size:
                leftovers[j] -= order
                break
            elif j == len(leftovers) - 1:
                leftovers.append(STOCK_SIZE)
                leftovers[-1] -= order
                break
    
    leftovers.sort(reverse = True)
    
    pos = None
    for i, leftover in enumerate(leftovers):
        if leftover == STOCK_SIZE:
            pos = i
        else:
            break
    if pos is not None and pos+1 < len(leftovers):
        leftovers = leftovers[pos+1:]
    
    result = 0    
    for i, leftover in enumerate(leftovers):
        result += (i+1) * leftover
    
    return result, len(leftovers)

In [37]:
def make_small_change(orders, num_orders):
    idxs = [i for i in range(num_orders)]
    random_length = random.randrange(1, num_orders)
    chosen = random.sample(idxs, k = random_length)
    for i in range(random_length):
        for j in range(chosen[i] + 1, num_orders):
            if orders[chosen[i]] != orders[j]:
                tmp = orders[chosen[i]]
                orders[chosen[i]] = orders[j]
                orders[j] = tmp
                break
    

In [38]:
def initialize(orders): # l je lista porudzbina tkd ako dobijem 2 od ove velicine i 3 od ove i blabla
    #onda treba sve to u jedan niz
    orders.sort(reverse = True) #nisam sig bas dal l da sortiram

In [39]:
# nk - The number of the ordered small items by the length of l k .
# N - The number of all small items.
# li - The length of the i th ordered item.
# M - The number of all large objects. - moze da ne bude dato u zadatku vec da se izracuna u initialize f-ji
# Lj - The length of the j th large object.
# wj - The trim loss appeared on j th large object.
# Xij - A binary variable which is equal to one if the i th Item is cut from the j th large object and
# zero otherwise.
# wj = Lj − for i in range(N) for j in range(M)(li * xij) - calc trim loss 
# 0 <= wj <= Lj - logicno

# 0 < j < M - materijal koji imamo
# 0 < i < N - broj porucenih 

In [40]:
import math

In [57]:
def simulated_annealing(orders, stocks, num_orders, num_stocks):
    stocks.sort(reverse = True)
    solution = deepcopy(orders)
    #initialize(solution)
    value, stocks_used = calc_solution_value(solution, stocks)
    best_solution = deepcopy(solution)
    best_value = value
    best_stocks_used = stocks_used
    T = 500
    a = 0.9
    min_temperature = 1e-3  

    
    while T > 1e-3:
        new_solution = deepcopy(solution)
        make_small_change(new_solution, num_orders)
        new_value, new_stocks_used = calc_solution_value(new_solution, stocks)
        
        diff = new_value - value
        if diff < 0:
            solution = deepcopy(new_solution)
            value = new_value
            stocks_used = new_stocks_used
            if new_value < best_value:
                best_value = new_value
                best_solution = deepcopy(new_solution)
                best_stocks_used = new_stocks_used
        else:
            if (math.e)**((-diff) / T):
                value = new_value
                solution = deepcopy(new_solution)
                stocks_used = new_stocks_used
        T = a * T
        
    return best_solution, best_value, best_stocks_used

In [58]:
def calc_num_stocks(orders, num_orders):
    stocks = [STOCK_SIZE]
    num_stock = 1
    for i in range(num_orders):
        for j in range(num_stock):
            if stocks[j] >= orders[i]:
                stocks[j] -= orders[i]
            else:
                if j == (num_stock - 1):
                    num_stock += 1
                    stocks.append(STOCK_SIZE)
                    stocks[num_stock - 1] -= orders[i]
    return num_stock

In [59]:
orders = [20, 30, 50, 10, 5, 5, 80, 35, 25, 40, 8, 12, 33, 27, 20, 90, 10, 90, 2, 3, 4, 1]
random.shuffle(orders)

In [60]:
num_orders = len(orders)
num_stocks = calc_num_stocks(orders, num_orders)
stocks = [100 for i in range(num_stocks)]

In [61]:
solution, value, stocks_used = simulated_annealing(orders, stocks, num_orders, num_stocks)
print(solution, value, stocks_used)

[33, 4, 1, 12, 27, 30, 50, 40, 35, 80, 90, 5, 2, 25, 20, 3, 5, 20, 10, 90, 8, 10] 0 6


In [62]:
orders = read_instance('test_instances/test_instance_20.csp')
num_orders = len(orders)
num_stocks = calc_num_stocks(orders, num_orders)
stocks = [100 for i in range(num_stocks)]

In [63]:
num_stocks

758

In [65]:
solution, value, stocks_used = simulated_annealing(orders, stocks, num_orders, num_stocks)
print(solution, value, stocks_used)

[48, 80, 90, 81, 60, 94, 81, 53, 32, 70, 92, 62, 38, 78, 84, 95, 77, 15, 18, 5, 50, 28, 26, 19, 46, 94, 78, 63, 66, 64, 82, 83, 45, 79, 71, 41, 84, 90, 45, 58, 46, 64, 47, 45, 45, 86, 53, 72, 32, 62, 32, 20, 79, 61, 93, 52, 52, 53, 69, 19, 42, 80, 23, 64, 69, 63, 94, 81, 63, 77, 46, 13, 32, 80, 66, 51, 86, 38, 56, 62, 10, 93, 66, 78, 63, 63, 36, 10, 54, 62, 51, 28, 85, 18, 40, 88, 68, 37, 16, 39, 41, 83, 69, 48, 43, 5, 52, 34, 90, 47, 27, 93, 58, 27, 24, 33, 81, 38, 86, 32, 40, 62, 36, 77, 85, 26, 60, 82, 87, 53, 57, 42, 88, 85, 39, 41, 75, 85, 43, 91, 55, 52, 71, 85, 24, 84, 76, 31, 91, 57, 81, 33, 6, 38, 45, 13, 53, 21, 26, 87, 31, 65, 51, 81, 70, 65, 22, 44, 80, 61, 26, 52, 39, 84, 33, 95, 67, 79, 66, 89, 25, 8, 27, 67, 92, 59, 70, 62, 19, 6, 52, 28, 81, 49, 92, 84, 73, 86, 24, 13, 74, 78, 39, 16, 18, 57, 48, 62, 87, 83, 67, 90, 52, 9, 95, 90, 31, 15, 70, 94, 13, 76, 79, 27, 64, 20, 10, 54, 67, 33, 43, 68, 58, 22, 74, 94, 7, 41, 45, 78, 76, 83, 68, 94, 27, 67, 49, 17, 46, 55, 47, 64